In [2]:
import face_recognition
import cv2
import os
import numpy as np

# Step 1: Load known faces from the student faces database
def load_known_faces(known_faces_folder):
    known_faces = []
    known_names = []
    
    for filename in os.listdir(known_faces_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Load the image and encode it
            image_path = os.path.join(known_faces_folder, filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)
            if encoding:  # Ensure encoding exists
                known_faces.append(encoding[0])
                known_names.append(os.path.splitext(filename)[0])  # Use filename (without extension) as name
    
    return known_faces, known_names

# Step 2: Detect faces and mark attendance
def detect_and_mark_faces(image_path, known_faces, known_names, tolerance=0.4):
    try:
        # Load the uploaded image using face_recognition
        image = face_recognition.load_image_file(image_path)
        
        # Find all face locations and encodings in the uploaded image
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)
        
        # Convert the image to BGR color for OpenCV to display correctly
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        attendance = []

        # Loop through each face encoding and location
        for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
            # Compute face distances from known faces
            face_distances = face_recognition.face_distance(known_faces, face_encoding)
            
            # Get the best match index based on the smallest distance
            best_match_index = np.argmin(face_distances)

            # Check if the closest match is below the tolerance threshold
            if face_distances[best_match_index] < tolerance:
                name = known_names[best_match_index]
                attendance.append(name)
            else:
                name = "Unknown"

            # Draw a rectangle around each face and put the name
            cv2.rectangle(image_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(image_bgr, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        print(f"Detected faces: {attendance}")
        
        return image_bgr, attendance
    
    except Exception as e:
        print(f"Error: {e}")
        return None, []

# Step 3: Main function to load the image, detect faces, and mark them
def upload_and_mark_faces(image_path, known_faces_folder, output_folder, tolerance=0.4):
    known_faces, known_names = load_known_faces(known_faces_folder)
    marked_image, attendance = detect_and_mark_faces(image_path, known_faces, known_names, tolerance)
    
    if marked_image is not None:
        # Save the output image with marked faces
        output_image_path = os.path.join(output_folder, "marked_faces.jpg")
        cv2.imwrite(output_image_path, marked_image)
        
        print(f"Marked image saved at: {output_image_path}")
        return attendance  # Return the attendance list

# Example: Replace with your image, known faces folder path, and output folder path
if __name__ == "__main__":
    image_path = "IMG_5945.jpg"  # Path of the uploaded image
    known_faces_folder = "student_faces"  # Path of the folder containing student face images
    output_folder = "output_images"  # Path of the folder where you want to save the marked image
    
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Set tolerance for face matching (lower value = stricter matching)
    tolerance = 0.4
    
    attendance_list = upload_and_mark_faces(image_path, known_faces_folder, output_folder, tolerance)
    
    print(f"Attendance marked: {attendance_list}")


Detected faces: ['persons face']
Marked image saved at: output_images/marked_faces.jpg
Attendance marked: ['x person']
